Loading Data

In [19]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
articles = pd.read_csv('articles.csv')

Preprocessing Data

In [20]:
articles.head()

,Unnamed: 0,title,date,stock
0,0.0,Stocks That Hit 52-Week Highs On Friday,2020-06-05 10:30:00-04:00,A
1,1.0,Stocks That Hit 52-Week Highs On Wednesday,2020-06-03 10:45:00-04:00,A
2,2.0,71 Biggest Movers From Friday,2020-05-26 04:30:00-04:00,A
3,3.0,46 Stocks Moving In Friday's Mid-Day Session,2020-05-22 12:45:00-04:00,A
4,4.0,B of A Securities Maintains Neutral on Agilent...,2020-05-22 11:38:00-04:00,A


In [21]:
import nltk
from datetime import datetime as dt
import pytz
from nltk.corpus import stopwords
nltk.download('stopwords')

# Credit to / help from https://saturncloud.io/blog/how-to-remove-stop-words-from-a-pandas-dataframe-using-python/
def remove_stopwords(words_tokenized):
    stop_words = set(stopwords.words('english'))  # List of english stopwords
    return [word for word in words_tokenized if word not in stop_words] # Using list comprehension, only choose the words that aren't stopwords

def convert_to_datetime(date_string):
    
    date_string, _ = date_string.rsplit("-", 1)
    
    converted_date = dt.strptime(date_string, '%Y-%m-%d %H:%M:%S')
    return converted_date

def preprocess(df, sample_size=None):
    df = df.dropna()
    # sample if specified
    if sample_size:
        df = df.sample(sample_size)
    # remove uncessary index column
    df = df.drop(df.columns[0], axis=1)
    # change stock column name to ticker
    df.rename(columns={'stock': 'ticker'}, inplace=True)
    # convert headlines to lowercase
    df['title'] = df['title'].str.lower()
    # remove punctuation
    df['title'] = df['title'].str.replace(r'[^a-zA-Z\s$0-9]', '', regex=True)
    # tokenize
    df['title'] = df['title'].str.split() 
    # remove stopwords
    df['title'] = df['title'].apply(remove_stopwords)
    # convert to datetime object
    df['date'] = df['date'].apply(convert_to_datetime)
    return df

articles = preprocess(articles, sample_size=1000)

articles

[nltk_data] Downloading package stopwords to C:\Users\Yashvardhan
[nltk_data]     Sharma\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,title,date,ticker
855979,"[macquarie, maintains, outperform, vail, resor...",2016-03-11 08:06:00,MTN
652509,"[imperva, option, alert, nov, 17, $45, calls, ...",2017-10-31 15:53:00,IMPV
73892,"[deutsche, bank, maintains, autonation, sell, ...",2012-10-26 08:06:00,AN
252518,"[euro, tech, holdings, says, presentation, sis...",2018-11-06 11:47:00,CLWT
745316,"[homebuilders, lifted, pending, sales, data]",2015-03-30 13:33:00,LEN
...,...,...,...
1044317,"[twelve, 12, niche, etfs, watch, new, year]",2011-12-13 21:31:00,QQQ
632207,"[option, alert, iac, interactive, put, volume,...",2012-05-02 12:11:00,IACI
1122771,"[investors, pay, attention, 27th, annual, pipe...",2015-12-02 11:54:00,SHPG
283152,"[benzingas, top, upgrades, downgrades, august,...",2018-08-10 09:17:00,CPB


Grab Stock Returns

Based on the time of the article published, we will retrieve two adjusted close prices of the stock and compute the corresponding return.

If the time of the article is published before 4:00 P.M. (non-inclusive), then:
1. The 'before' price will be the most recent (before the date) trading day's adjusted close price
2. The 'after' price will be the most upcoming trading day's adjusted close price

If the time of the article is published after 4:00 P.M., then:
1. The 'before' price will be the same day's adjusted close price
2. The 'after' priec will be the next day's adjusted close price

In [22]:
import pandas_market_calendars as mcal
from datetime import timedelta
# The paramater forward is a boolean representing whether we are looking for the next valid trading day or the most recent trading day
def getValidTradingCloseDate(date, forward=True):
        nyse = mcal.get_calendar('NYSE')
        if forward:
            start_date = date
            end_date = date+timedelta(days=15)
        else:
            start_date = date-timedelta(days=15)
            end_date = date

        validTradingDays = nyse.valid_days(start_date=start_date , end_date=end_date)
        return validTradingDays.date[2] if forward else validTradingDays.date[-2]


In [23]:
#Get all the yfinance data we need based on date.
import yfinance as yf

def retrieve_yfinance_data(row):
    curr_date = row['date']
    
    eod = dt.strptime('16:00:00', '%H:%M:%S').time()
    
    if curr_date.time() > eod:
        start_date = curr_date.date()
        end_date = getValidTradingCloseDate(start_date, forward=True)
    else:
        end_date = curr_date.date()
        start_date = getValidTradingCloseDate(end_date, forward=False)
        end_date = end_date + timedelta(days=1)
        
    data = yf.download(row['ticker'], start=start_date, end=end_date, progress=False, show_errors=False)
    
    if len(data) > 0:
        returns = (data['Adj Close'][-1] - data['Adj Close'][0]) / data['Adj Close'][0]
        return returns
    else:
        return None

In [24]:
from tqdm import tqdm

def get_returns(df):
    # df['returns'] = df.apply(retrieve_yfinance_data, axis=1)
    for idx, row in tqdm(df.iterrows(), total=df.shape[0]):
        returns = retrieve_yfinance_data(row)
        # print(returns)
        df.loc[idx,'returns'] = returns

    return df

articles = get_returns(articles)

100%|██████████| 1000/1000 [10:13<00:00,  1.63it/s]


In [25]:
articles

,title,date,ticker,returns
855979,"[macquarie, maintains, outperform, vail, resor...",2016-03-11 08:06:00,MTN,0.027351
652509,"[imperva, option, alert, nov, 17, $45, calls, ...",2017-10-31 15:53:00,IMPV,0.027678
73892,"[deutsche, bank, maintains, autonation, sell, ...",2012-10-26 08:06:00,AN,-0.025131
252518,"[euro, tech, holdings, says, presentation, sis...",2018-11-06 11:47:00,CLWT,0.000000
745316,"[homebuilders, lifted, pending, sales, data]",2015-03-30 13:33:00,LEN,0.021019
...,...,...,...,...
1044317,"[twelve, 12, niche, etfs, watch, new, year]",2011-12-13 21:31:00,QQQ,-0.015603
632207,"[option, alert, iac, interactive, put, volume,...",2012-05-02 12:11:00,IACI,NaN
1122771,"[investors, pay, attention, 27th, annual, pipe...",2015-12-02 11:54:00,SHPG,-0.002839
283152,"[benzingas, top, upgrades, downgrades, august,...",2018-08-10 09:17:00,CPB,-0.021996


In [26]:
articles1 = articles.dropna()

In [28]:
articles1.to_pickle('699_processed_data.pkl')